In [31]:
from openai import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import SystemMessage
import numpy as np
from IPython.display import display, Markdown
from dotenv import load_dotenv
import time
import os
from pinecone import Pinecone 
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore

In [32]:
load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key)
client = OpenAI()

In [33]:
index_name = "ia-sensor"

if not pc.has_index(index_name):
   pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

In [34]:
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

vectorstore = PineconeVectorStore.from_documents(
    documents=split_texts, 
    embedding=embeddings, 
    index_name=index_name 
)


#Retriver
retriever = vectorstore.as_retriever()
# Pegar os documentos relevantes baseado na pergunta
pergunta = "Como faço para congifurar a central para o cliente?"
docs_relevantes = retriever.invoke(pergunta)

# Preparar o contexto
context_list = []
for doc in docs_relevantes:
    context_list.append(doc.page_content)

context = "\n\n".join(context_list)

# Instanciando o modelo de IA e enviando a pergunta
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", 
         "content": f"Use apenas as informações abaixo para responder a pergunta do usuário. Seja detalhado, em formato de passo a passo.\n\n{context}"
         },
        {"role": "user", 
        "content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"
         } 
    ]
)

# Resposta
display(Markdown(completion.choices[0].message.content))


Para configurar a central para o cliente, seguindo o padrão apresentado acima, siga os passos detalhados a seguir:

1. Acesse a central via PUTTY.

2. Execute o comando para iniciar a configuração da central:
   
   ```
   sensorgw appConfig
   ``` 
   - Responda se é uma configuração de servidor padrão (Default server configuration) pressionando "y".
   - Indique o nome da aplicação web do cliente (Client WebApp), por exemplo, "TESTE".
   - Preencha as informações solicitadas para o cliente: Client Login, Client Password, CheckConnection, CheckModem, e o nome da operadora (Carrier).

3. Execute os comandos para sincronizar e parar a central:
   
   ```
   sync
   sensorgw stop
   ```

4. Realize a configuração do serviço HAMACHI:

   - Pare o serviço do Hamachi:
     ```
     /etc/init.d/logmein-hamachi stop
     ```

   - Remova as configurações prévias do Hamachi:
     ```
     rm -Rf /var/lib/logmein-hamachi/
     ```

   - Inicie o serviço do Hamachi:
     ```
     /etc/init.d/logmein-hamachi start
     ```

5. Realize o login no Hamachi:
   
   ```
   hamachi login
   ```

6. Defina o nome da central para o Hamachi:
   
   ```
   hamachi set-nick “CLIENTE”
   ```

7. Vincule a conta do Hamachi:
   
   ```
   hamachi attach eduardosleal@mcasistemas.com.br
   ```

8. Conecte-se à conta do Hamachi:
   
   ```
   hamachi do-join 158-893-675
   ```

   - Será solicitada a senha, que no caso é "mcaUser".

Dessa forma, seguindo esses passos, você conseguirá configurar a central para envio de dados ao portal do cliente de acordo com as instruções fornecidas no contexto.